# Correlação de Pearson – detecção dinâmica de colunas numéricas
Este notebook resolve o erro **KeyError** ao tentar acessar colunas que não
existem no DataFrame (`df[numeric_cols]`). Ele faz:
1. Carregamento do dataset `uf_votos_ideologia_socioeco_2022.csv` (ajuste caminho se necessário).
2. Verificação de quais colunas da lista-alvo realmente existem no arquivo.
3. Conversão segura de strings com vírgula em `float`.
4. Cálculo das correlações de Pearson e visualização.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

csv_path = Path('uf_votos_ideologia_socioeco_2022.csv')  # ajuste se necessário

## 1. Carregar CSV

In [ ]:
df = pd.read_csv(csv_path)
print('Colunas encontradas:', list(df.columns))
df.head()

## 2. Definir lista-alvo e filtrar apenas colunas existentes

In [ ]:
target_numeric = [
    'PIB_per_capita','PIB_milhoes','IDHM_2021','Gini',
    'Esperanca_vida_anos','Mort_infantil_pmil','Analfabetismo_perc','IPCA_2022_perc',
    'esquerda_perc','centro-esquerda_perc','centro_perc','centro-direita_perc','direita_perc'
]

numeric_cols = [c for c in target_numeric if c in df.columns]
print('Colunas numéricas detectadas:', numeric_cols)

## 3. Converter vírgulas para ponto e forçar float

In [ ]:
for col in numeric_cols:
    df[col] = (df[col].astype(str)
                      .str.replace(',', '.', regex=False)
                      .str.replace('%', '', regex=False))
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Verificar dtypes
df[numeric_cols].dtypes

## 4. Criar blocos voto esquerda/direita

In [ ]:
df['left_vote'] = df.get('esquerda_perc', 0) + df.get('centro-esquerda_perc', 0)
df['right_vote'] = df.get('direita_perc', 0) + df.get('centro-direita_perc', 0)

## 5. Calcular correlações

In [ ]:
variables = [v for v in ['PIB_per_capita','PIB_milhoes','IDHM_2021','Gini',
                         'Esperanca_vida_anos','Mort_infantil_pmil','Analfabetismo_perc','IPCA_2022_perc'] if v in df.columns]

corr_rows = []
for var in variables:
    corr_right = df[[var, 'right_vote']].corr().iloc[0, 1]
    corr_left = df[[var, 'left_vote']].corr().iloc[0, 1]
    corr_rows.append({'variavel': var, 'corr_direita': corr_right, 'corr_esquerda': corr_left})

corr_df = pd.DataFrame(corr_rows).sort_values('corr_direita', key=lambda s: s.abs(), ascending=False)
corr_df

## 6. Heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(6, len(corr_df)*0.5+1))
heat_data = corr_df[['corr_esquerda', 'corr_direita']].to_numpy()
img = ax.imshow(heat_data, aspect='auto', cmap='viridis')
ax.set_xticks([0,1]); ax.set_xticklabels(['Esquerda','Direita'])
ax.set_yticks(range(len(corr_df))); ax.set_yticklabels(corr_df['variavel'])
plt.colorbar(img, ax=ax)
plt.title('Correlação de Pearson (UF – 2022)')
plt.tight_layout()
plt.show()